<a href="https://colab.research.google.com/github/ghasemieh/Duplicate-Bug-Identification-System/blob/master/Model_1_V03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Model-1-V3

In [0]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [0]:
data = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')

In [0]:
data.head()

,id,product,component,reporter,bug_status,resolution,priority,bug_severity,version,short_desc,opendate,dup_list,root_id,disc_id
0,2,Log4j -,Layout,bugzilla,CLOS,FIXE,P3,nor,unspe,Just testing the Boogzeela setup for log4j,2001-01-08,NaN,NaN,NaN
1,3,Log4j -,Appender,matthew_scully,RESO,FIXE,P3,nor,unspe,file sharing,2001-01-08,NaN,NaN,NaN
2,22,Apache h,All,greenrd,CLOS,WONT,P5,enh,2.0-H,Need hooks for user-defined error handling,2001-01-09,NaN,NaN,NaN
3,27,Log4j -,Layout,richard.mccarthy,RESO,INVA,P1,nor,1.0,Error in the formatting of the (%F:%L) layout ...,2001-01-10,NaN,NaN,NaN
4,29,Log4j -,Other,woge,RESO,FIXE,P3,nor,1.0,"Calling Category.error(Object, Throwable) thro...",2001-01-11,NaN,NaN,NaN


**Remove NaN from the short_desc**

In [0]:
data.isnull().sum()

id                  0
product             0
component           0
reporter            0
bug_status          0
resolution          0
priority            0
bug_severity        0
version             0
short_desc         58
opendate            0
dup_list        41531
root_id         38016
disc_id         38016
dtype: int64

In [0]:
df = data[['id','product','short_desc','dup_list']]
df.head()

,id,product,short_desc,dup_list
0,2,Log4j -,Just testing the Boogzeela setup for log4j,NaN
1,3,Log4j -,file sharing,NaN
2,22,Apache h,Need hooks for user-defined error handling,NaN
3,27,Log4j -,Error in the formatting of the (%F:%L) layout ...,NaN
4,29,Log4j -,"Calling Category.error(Object, Throwable) thro...",NaN


In [0]:
df.shape

(44049, 4)

Remove Blank Records

In [0]:
blanks = []

for i,id,pro,sd,dl in df.itertuples():
    if type(sd)==str:
        if sd.isspace():
            blanks.append(i)
    else:
        blanks.append(i)
len(blanks)  

58

In [0]:
df.drop(blanks,inplace=True)
df.isnull().sum()

id                0
product           0
short_desc        0
dup_list      41474
dtype: int64

**Similarity Methods:**

In [0]:
#conda install -c conda-forge spacy
#conda install -c conda-forge spacy-lookups-data
#!python -m spacy download en_core_web_lg

In [0]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

In [0]:
def tokenize(df,id):
    s = df[df['id'] == id]
    s = s.short_desc.to_string(index=False)
    s = s[1:]
    # print ('\n'+ s)
    doc = nlp(s)
    #for token in doc:
    #    print(token.text, end=' | ')
    #displacy.render(doc, style='dep', jupyter=True, options={'distance': 110})    
    return doc

In [0]:
doc1 = tokenize(df,355)
doc2 = tokenize(df,475)

**Remove Puntuations**

In [0]:
def show_lemmas(text):
    for token in text:
        print(f'{token.text:{20}} {token.pos_:{6}} {token.lemma:<{22}} {token.lemma_}')

In [0]:
#Remove Puntuations
def remove_puntuation(doc):
    #show_lemmas(doc)
    s = []
    for token in doc:
        if token.is_punct == False:
            s.append(token) 
    #Convert list to string
    str1 = ""    
    for ele in s:  
        str1 += str(ele) +' '     
    return nlp(str1[:-1]) #return doc 

In [0]:
doc1 = remove_puntuation(doc1)

In [0]:
doc2 = remove_puntuation(doc2)

In [0]:
#show_lemmas(doc1)

**Remove Stop Words**

In [0]:
#Remove Stop Words
def remove_stop_word(doc): # accept string
    s = []
    for token in doc:
        if token.is_stop == False:
            s.append(token) 
    #Convert list to string
    str1 = ""    
    for ele in s:  
        str1 += str(ele) +' '     
    return nlp(str1[:-1]) #return doc 

In [0]:
doc1 = remove_stop_word(doc1)
doc1.text

'request.getCookies work BugRat Repor'

In [0]:
doc2 = remove_stop_word(doc2)
doc2.text

'SimpleTcpConnector work BugRat Report#802'

In [0]:
print('Similarity: ' + str(doc1.similarity(doc2)))

Similarity: 0.9999999906303305


**Build a function of all previous work**

In [0]:
# Put all prevouse work together in a function
def prep_func(df,id):
    s = tokenize(df,id)
    s = remove_puntuation(s)
    doc = remove_stop_word(s)
    return doc

In [0]:
# Prepare the vectors for each document
import time

start_time = time.time()
c = 0
prepared = []
l = len(df.index)

for i,id,pro,sd,db in df.itertuples():
    c = c + 1
    doc = prep_func(df,id)
    prepared.append((id,pro,doc))
    
    # Show the progress in the output
    c = c + 1
    if c % 100 == 0:
        progress = c/l*100
        print('Progress: %s %%' % round(progress, 2), end="\r", flush=True)
    if c > 1000:
        print('\nDone')
        break
print("--- %s seconds ---" % (time.time() - start_time))  # show the time of process

Progress: 2.27 %
Done
--- 13.354298114776611 seconds ---


In [0]:
c = 0
for i in prepared:
    print(i[0], '\t',i[1], '\t', i[2].text)
    c = c + 1
    if c > 5:
        break

2 	 Log4j - 	 testing Boogzeela setup log4j
3 	 Log4j - 	 file sharing
22 	 Apache h 	 Need hooks user defined error handling
27 	 Log4j - 	 Error formatting F:%L layout
29 	 Log4j - 	 Calling Category.error(Object Throwable thro
31 	 Log4j - 	 Conversion log4j Priority types NT Event


In [0]:
# Calculate one by one similarity score
# x[0]: id , x[1]: product, x[2]: short_desc

start_time = time.time()
similarities = []
l = len(prepared)
c = 0

for i in prepared:
    c = c + 1
    for j in prepared:
        if i[1] == j[1]: # if two bug reports belong to the same product then check the similarity
            score = i[2].similarity(j[2])
            similarities.append((i[0],j[0],score))
            
    # Show the progress in the output        
    if c % 10 == 0:
        progress = c/l*100
        print('Progress: %s %%' % round(progress, 1), end="\r", flush=True)
print('\nDone')
print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process

Progress: 99.8 %
Done
--- 73.9271948337555 seconds ---


In [0]:
# Sort based on the score
sorted_similarities = sorted(similarities, key=lambda tup: tup[2], reverse=True)

In [0]:
sorted_similarities[510:530]

[(566, 566, 1.0),
 (567, 567, 1.0),
 (569, 569, 1.0),
 (355, 475, 0.9999999906303305),
 (475, 355, 0.9999999906303305),
 (373, 441, 0.9999999741066152),
 (441, 373, 0.9999999741066152),
 (317, 341, 0.9673615744389031),
 (341, 317, 0.9673615744389031),
 (144, 145, 0.955040353250707),
 (145, 144, 0.955040353250707),
 (101, 483, 0.9373680380808607),
 (483, 101, 0.9373680380808607),
 (44, 66, 0.9349526529218797),
 (66, 44, 0.9349526529218797),
 (45, 71, 0.9278628880921946),
 (71, 45, 0.9278628880921946),
 (94, 426, 0.893976882234901),
 (426, 94, 0.893976882234901),
 (291, 315, 0.8936179344666283),
 (315, 291, 0.8936179344666283),
 (185, 293, 0.8873870969586577),
 (293, 185, 0.8873870969586577),
 (152, 267, 0.8863471569584803),
 (267, 152, 0.8863471569584803),
 (86, 181, 0.8858457011580854),
 (181, 86, 0.8858457011580854),
 (270, 527, 0.8780640978312688),
 (527, 270, 0.8780640978312688),
 (452, 453, 0.8777453865602048),
 (453, 452, 0.8777453865602048),
 (284, 315, 0.8694118939450673),
 (315

**Test the outputs**

In [0]:
def test(df,id1,id2):
    d1 = tokenize(df,id1)
    print('s1: ' + d1.text)
    d2 = tokenize(df,id2)
    print('s2: ' + d2.text)
    d1 = remove_puntuation(d1)
    d2 = remove_puntuation(d2)
    doc1 = remove_stop_word(d1)
    print('Doc1: ' + doc1.text + '\t')
    doc2 = remove_stop_word(d2)
    print('Doc2: ' + doc2.text + '\t')
    print('Similarity: ' + str(doc1.similarity(doc2)))

In [0]:
test(df,355, 475)

s1: request.getCookies() doesn't work BugRat Repor...
s2: SimpleTcpConnector doesn't work BugRat Report#802
Doc1: request.getCookies work BugRat Repor	
Doc2: SimpleTcpConnector work BugRat Report#802	
Similarity: 0.9999999906303305


**Assessment of the accuracy**